# GDVSpectra Likelihood module tests

## Globals

In [1]:
# set globals paths
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [2]:
# imports
import os
import random
import numpy as np
import xarray as xr
from IPython.utils import io

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import full arcpy toolbox
arcpy.ImportToolbox(r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

<module 'toolbox'>

### Reload libraries

In [3]:
# if scripts change, reload
from importlib import reload
reload(test_funcs)

<module 'test_funcs' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\tests\\code\\test_funcs.py'>

### Set data files and locations

In [4]:
# setup general io. nc ins and outs exist in these folders
input_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs'
output_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\outputs'

# temp nc file for use when breaking ncs
temp_nc = os.path.join(input_folder, 'temp_nc.nc')  

# setup landsat cubes paths
ls_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_1_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_ls_90_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_ls_90_20_raw_odc.nc",
]

# setup sentinel2 cubes paths
s2_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_1_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_3_s2_16_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_4_s2_16_20_raw_odc.nc",
]

### Set specific raw netcdf file

In [35]:
# set specific dataset
nc_file = ls_cubes[1]

### Set up function to iterate corruptor and tests

In [6]:
def run_corruptors_through_tests(in_nc, nc_coors, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed.
    """
    
    for nc_corr in nc_corrs:
        corr_name = nc_corr[0].__name__                   # name of current corruptor func
        corr_func, corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params

        # notify
        print('Corrupting NetCDF via: {}.\n'.format(corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            corr_func(**corr_params)
        except Exception as e:    
            print(e)
            print('Corruptor did not have enough data to work with. Skipping.\n')

        # iter each test func and apply to current corrupt nc
        for test in tests:
            test_nc_name = corr_name + '_' + test[0]    # name of current test nc
            test_func, test_params = test[1], test[2]   # pointer to test func and dict of params
            test_msg = test[3]
            
            # notify of test message
            print(test_msg)

            # create output nc file path and name and update params for in/out paths
            out_nc_file = os.path.join(output_folder, test_nc_name)
            
            # remove output nc if exists
            if os.path.exists(out_nc_file):
                os.remove(out_nc_file)
            
            # update params
            test_params.update({'in_nc_file': temp_nc, 'out_likelihood_nc_file': out_nc_file})

            # perform current test
            try:
                # notify
                print('Performing test: {}.'.format(test_nc_name))

                # perform test, provide prints if requested
                if not verbose:
                    with io.capture_output() as cap:
                        test_func(**test_params)
                else:
                    test_func(**test_params)
                    print('\n')

            except Exception as e:    
                print(e)

        # notify
        print('All tests applied to corruptor NetCDF.\n\n')

### Set up netcdf corruptor functions

In [7]:
# these are numerous netcdf corruptors. feed a raw nc in, break it, output as temp nc
# comment out any that are irrelevant
# each of these uncommented will be fed through the tests below
def build_nc_corruptors(temp_nc):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    cs = []

    # func: raw default dataset, no changes
    cs.append([test_funcs.nc_default, {'in_nc': temp_nc}])

    # func: remove x, y, time, spatial_ref coords
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'x'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'y'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'time'}])
    #cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'spatial_ref'}])

    #func: remove red and oa_fmask band vars
    #cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'nbart_red'}])
    #cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'oa_fmask'}])

    #func: limit number of years in various combos
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1990, 'e_year': 1990}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2010, 'e_year': 2010}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2012, 'e_year': 2012}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1991, 'e_year': 1992}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2005, 'e_year': 2006}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2019, 'e_year': 2020}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1993, 'e_year': 1995}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2010, 'e_year': 2012}])
    #cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2011, 'e_year': 2013}])

    #func: set all vars to nan
    #cs.append([test_funcs.set_nc_vars_all_nan, {'in_nc': temp_nc}])

    #func: set all vars to zero
    #cs.append([test_funcs.set_nc_vars_all_zero, {'in_nc': temp_nc}])

    #func: set all vars for 10 rand times to all nan
    #cs.append([test_funcs.set_nc_vars_random_all_nan, {'in_nc': temp_nc, 'num': 10}])

    #func: strip all attrs from nc    
    #cs.append([test_funcs.strip_nc_attributes, {'in_nc': temp_nc}])

    #func: set vars in first and last time index to all nan
    #cs.append([test_funcs.set_end_times_to_all_nan, {'in_nc': temp_nc}])

    #func: reduce whole nc to one random time slice
    #cs.append([test_funcs.reduce_to_one_scene, {'in_nc': temp_nc}])

    #func: set wet months all nan, all years, for specific months
    #cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1]}])
    #cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1, 2, 3]}])
    #cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]}])

    #func: set wet months all nan, specific years, for specific months
    #cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [1990], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2005], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2006, 2007], 'months': [1, 2, 3]}])

    #func: drop wet months, all years, for specific months
    #cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1]}])
    #cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1, 2, 3]}])
    #cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [7, 8, 9, 10, 11, 12]}])

    #func: drop wet months, specific years, for specific months (note: seperate tests)
    #cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [1990], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2009], 'months': [1, 2, 3]}])
    #cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2007, 2008], 'months': [1, 2, 3]}])

    #func: remove crs attribute
    #cs.append([test_funcs.remove_crs_attr, {'in_nc': temp_nc}])

    #func: invalidate crs attribute
    #cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': 'EPSG:4326'}])
    #cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': ''}])

    #func: remove nodatavals attribute
    #cs.append([test_funcs.remove_nodatavals_attr, {'in_nc': temp_nc}])
    
    return cs

nc_corruptors = build_nc_corruptors(temp_nc=temp_nc)

## Run tests

### Test One: Wet Months

In [73]:
def build_test_one_funcs(in_nc, temp_nc):
    """sets up test one functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '',                      # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []

    # func: default wet months 1, 2, 3
    msg = 'Running Test One: default wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': '1;2;3'})
    ts.append(['t_1_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: no wet months
    msg = 'Running Test One: "" wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': ''})
    ts.append(['t_1_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: one random month
    rand = str(random.randint(1, 5))
    msg = 'Running Test One: single random wet month ({}).'.format(rand)
    params = inputs.copy()
    params.update({'in_wet_months': rand})
    ts.append(['t_1_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: three months (2, 3, 4)
    msg = 'Running Test One: 2;3;4 as wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': '2;3;4'})
    ts.append(['t_1_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
    
    # func: all dry season months in wet season
    msg = 'Running Test One: dry months 5;6;7;8;9 as wet months.'
    params = inputs.copy()
    params.update({'in_wet_months': '5;6;7;8;9'})
    ts.append(['t_1_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: wet season includes a month in dec
    msg = 'Running Test One: wet months contain dec (12).'
    params = inputs.copy()
    params.update({'in_wet_months': '12;1;2'})
    ts.append(['t_1_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: wet season includes only one month in dec
    msg = 'Running Test One: single wet month in dec (12).'
    params = inputs.copy()
    params.update({'in_wet_months': '12'})
    ts.append(['t_1_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     
        
    return ts

### Test One: Run!

In [74]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_one_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

[<function nc_default at 0x0000019888E898B8>, {'in_nc': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc'}]
Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
'in_nc' is an invalid keyword argument for print()
Corruptor did not have enough data to work with. Skipping.

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\nc_default_t_1_def.nc', 'in_wet_months': '1;2;3', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: nc_default_t_1_def.nc.
{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Pack

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_coord_t_1_c.nc', 'in_wet_months': '2;3;4', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: remove_coord_t_1_c.nc.
{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_coord_t_1_d.nc', 'in_wet_months': '5;6;7;8;9', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_q

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_var_t_1_f.nc', 'in_wet_months': '12', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: remove_var_t_1_f.nc.
All tests applied to corruptor NetCDF.


[<function limit_years at 0x0000019888E95168>, {'in_nc': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 's_year': 1990, 'e_year': 1990}]
Corrupting NetCDF via: limit_years.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
'in_nc' is an invalid keyword argument for print()
Corruptor did not have enough data to work wit


{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\limit_years_t_1_b.nc', 'in_wet_months': '4', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: limit_years_t_1_b.nc.
{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\limit_years_t_1_c.nc', 'in_wet_months': '2;3;4', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\limit_years_t_1_e.nc', 'in_wet_months': '12;1;2', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: limit_years_t_1_e.nc.
{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\limit_years_t_1_f.nc', 'in_wet_months': '12', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0

...

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\set_specific_years_season_nan_t_1_a.nc', 'in_wet_months': '', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: set_specific_years_season_nan_t_1_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Wet month(s): Value is required
Failed to execute (GDVSpectra_Likelihood).

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\set_specific_years_seaso

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_all_specific_season_nan_t_1_b.nc', 'in_wet_months': '4', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: remove_all_specific_season_nan_t_1_b.nc.
{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_all_specific_season_nan_t_1_c.nc', 'in_wet_months': '2;3;4', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalu


[<function remove_all_specific_season_nan at 0x0000019888E95678>, {'in_nc': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'months': [7, 8, 9, 10, 11, 12]}]
Corrupting NetCDF via: remove_all_specific_season_nan.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
'in_nc' is an invalid keyword argument for print()
Corruptor did not have enough data to work with. Skipping.

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_all_specific_season_nan_t_1_def.nc', 'in_wet_months': '1;2;3', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}


{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_specific_years_season_nan_t_1_a.nc', 'in_wet_months': '', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: remove_specific_years_season_nan_t_1_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Wet month(s): Value is required
Failed to execute (GDVSpectra_Likelihood).

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\remove_specific_ye

{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\invalidate_crs_attr_t_1_c.nc', 'in_wet_months': '2;3;4', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05, 'in_fmask_flags': 'Valid;Snow;Water', 'in_max_cloud': 10, 'in_interpolate': True, 'in_add_result_to_map': True}
Performing test: invalidate_crs_attr_t_1_c.nc.
{'in_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\inputs\\temp_nc.nc', 'out_likelihood_nc_file': 'E:\\Curtin\\GDVII - General\\Work Package 2\\test_data\\gdvspectra_likelihood\\outputs\\invalidate_crs_attr_t_1_d.nc', 'in_wet_months': '5;6;7;8;9', 'in_dry_months': '9;10;11', 'in_veg_idx': 'MAVI', 'in_mst_idx': 'NDMI', 'in_zscore_pvalue': None, 'in_stand_quppe

### Test Two: Dry Months

In [80]:
def build_test_two_funcs(in_nc, temp_nc):
    """sets up test two functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                      # wet months 
        'in_dry_months': '',                      # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default dry months 9, 10, 11
    msg = 'Running Test Two: default dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': '9;10;11'})
    ts.append(['t_2_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: no dry months
    msg = 'Running Test Two: "" dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': ''})
    ts.append(['t_2_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: one random month
    rand = str(random.randint(7, 12))
    msg = 'Running Test Two: single random dry month ({}).'.format(rand)
    params = inputs.copy()
    params.update({'in_dry_months': rand})
    ts.append(['t_2_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: three months (8, 9, 10)
    msg = 'Running Test Two: 10;11;12 as dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': '10;11;12'})
    ts.append(['t_2_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
    
    # func: all dry season months in wet season
    msg = 'Running Test Two: dry months 1;2;3;4;5 as dry months.'
    params = inputs.copy()
    params.update({'in_dry_months': '1;2;3;4;5'})
    ts.append(['t_2_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: wet season includes a month in dec
    msg = 'Running Test Two: dry months contain one month in jan (1).'
    params = inputs.copy()
    params.update({'in_dry_months': '11;12;1'})
    ts.append(['t_2_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
        
    return ts

### Test Two: Run!

In [79]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_two_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Performing test: nc_default_t_1_def.nc.
Performing test: nc_default_t_1_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Dry month(s): Value is required
Failed to execute (GDVSpectra_Likelihood).

Performing test: nc_default_t_1_b.nc.
Performing test: nc_default_t_1_c.nc.
Performing test: nc_default_t_1_d.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

Performing test: nc_default_t_1_e.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_coord.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing coord: x
Performing test: remove_coord_t_1_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (GDVSpectra_Likelihood).

Performing test: remove_coo


Performing test: limit_years_t_1_c.nc.
Input NetCDF needs three or more years of data.
Failed to execute (GDVSpectra_Likelihood).

Performing test: limit_years_t_1_d.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

Performing test: limit_years_t_1_e.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: limit_years.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Limiting years to 2012 - 2012.
NetCDF: Invalid argument
Corruptor did not have enough data to work with. Skipping.

Performing test: limit_years_t_1_def.nc.
Input NetCDF must have x, y and time coords.
Failed to execute (GDVSpectra_Likelihood).

Performing test: limit_years_t_1_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Dry month(s): Value is required
Failed to execute (GDVSpectra_Likelihood).

Performing test: limit_years_t_1_b.nc.
Input NetCDF 


All tests applied to corruptor NetCDF.


Corrupting NetCDF via: set_nc_vars_all_zero.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Setting nc var data to all zero
Performing test: set_nc_vars_all_zero_t_1_def.nc.
NetCDF is completely full of 0s. Please download again.
Failed to execute (GDVSpectra_Likelihood).

Performing test: set_nc_vars_all_zero_t_1_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Dry month(s): Value is required
Failed to execute (GDVSpectra_Likelihood).

Performing test: set_nc_vars_all_zero_t_1_b.nc.
NetCDF is completely full of 0s. Please download again.
Failed to execute (GDVSpectra_Likelihood).

Performing test: set_nc_vars_all_zero_t_1_c.nc.
NetCDF is completely full of 0s. Please download again.
Failed to execute (GDVSpectra_Likelihood).

Performing test: set_nc_vars_all_zero_t_1_d.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

Performing test: set_nc_vars_all_zero_t_1_e.nc.
Canno


Performing test: remove_specific_years_season_nan_t_1_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Dry month(s): Value is required
Failed to execute (GDVSpectra_Likelihood).

Performing test: remove_specific_years_season_nan_t_1_b.nc.
No wet season months captured in NetCDF.
Failed to execute (GDVSpectra_Likelihood).

Performing test: remove_specific_years_season_nan_t_1_c.nc.
No wet season months captured in NetCDF.
Failed to execute (GDVSpectra_Likelihood).

Performing test: remove_specific_years_season_nan_t_1_d.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

Performing test: remove_specific_years_season_nan_t_1_e.nc.
Cannot use same value for wet and dry months.
Failed to execute (GDVSpectra_Likelihood).

All tests applied to corruptor NetCDF.


Corrupting NetCDF via: remove_specific_years_season_nan.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Removing season in years: [2007, 2008] to nan with mont

### Test Three: Vegetation Idx

In [87]:
def build_test_three_funcs(in_nc, temp_nc):
    """sets up test three functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': '',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default veg idx mavi
    msg = 'Running Test Three: default veg idx (mavi).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MAVI'})
    ts.append(['t_3_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is ''
    msg = 'Running Test Three: veg idx is "".'
    params = inputs.copy()
    params.update({'in_veg_idx': ''})
    ts.append(['t_3_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is ndvi
    msg = 'Running Test Three: veg idx is NDVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NDVI'})
    ts.append(['t_3_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is evi
    msg = 'Running Test Three: veg idx is EVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'EVI'})
    ts.append(['t_3_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is savi
    msg = 'Running Test Three: veg idx is SAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'SAVI'})
    ts.append(['t_3_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is msavi
    msg = 'Running Test Three: veg idx is MSAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MSAVI'})
    ts.append(['t_3_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: veg idx is slavi
    msg = 'Running Test Three: veg idx is SLAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'SLAVI'})
    ts.append(['t_3_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: veg idx is mavi
    msg = 'Running Test Three: veg idx is MAVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'MAVI'})
    ts.append(['t_3_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: veg idx is kndvi
    msg = 'Running Test Three: veg idx is kNDVI.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'kNDVI'})
    ts.append(['t_3_h.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: veg idx is tcg
    msg = 'Running Test Three: veg idx is TCG.'
    params = inputs.copy()
    params.update({'in_veg_idx': 'TCG'})
    ts.append(['t_3_i.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
    
    # func: veg idx is nrva (non-existant)
    msg = 'Running Test Three: veg idx is NRVA (non-existant).'
    params = inputs.copy()
    params.update({'in_veg_idx': 'NRVA'})
    ts.append(['t_3_j.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    
        
    return ts

### Test Three: Run!

In [88]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_three_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc
No changes, setting up for default dataset.
Performing test: nc_default_t_3_def.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [1, 2, 3, 9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['mavi', 'ndmi'].
Calculating ind

Masking similarity areas to higher vege and moist areas.
Calculated seasonal similarity successfully.
Generating groundwater-dependent vegetation (GDV) model.
Generated groundwater-dependent vegetation model successfully
Exporting xarray as netcdf file.
Exported xarray as netcdf successfully.


Performing test: nc_default_t_3_g.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [1, 2, 3, 9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['mav

### Test Four: Moisture Idx

In [89]:
def build_test_four_funcs(in_nc, temp_nc):
    """sets up test four functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': '',                         # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default mst idx ndmi
    msg = 'Running Test Four: default mst idx (ndmi).'
    params = inputs.copy()
    params.update({'in_mst_idx': 'NDMI'})
    ts.append(['t_4_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: mst idx is ''
    msg = 'Running Test Four: mst idx is "".'
    params = inputs.copy()
    params.update({'in_mst_idx': ''})
    ts.append(['t_4_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: mst idx is gvmi
    msg = 'Running Test Four: mst idx is gvmi.'
    params = inputs.copy()
    params.update({'in_mst_idx': 'GVMI'})
    ts.append(['t_4_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: mst idx is rasd
    msg = 'Running Test Four: mst idx is rasd.'
    params = inputs.copy()
    params.update({'in_mst_idx': 'RASD'})
    ts.append(['t_4_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])   
            
    return ts

### Test Four: Run!

In [90]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_four_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc
No changes, setting up for default dataset.
Performing test: nc_default_t_4_def.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [1, 2, 3, 9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['mavi', 'ndmi'].
Calculating ind

### Test Five: Outlier Correction

In [13]:
def build_test_five_funcs(in_nc, temp_nc):
    """sets up test five functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                         # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default zscore p-value
    msg = 'Running Test Five: default zscore p-value (None).'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': None})
    ts.append(['t_5_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: zscore p-value is ''
    msg = 'Running Test Five: zscore p-value is "".'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': ''})
    ts.append(['t_5_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
 
    # func: zscore p-value is string of 0.01
    msg = 'Running Test Five: zscore p-value is "0.01".'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': "0.01"})
    ts.append(['t_5_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: zscore p-value is string of 0.05
    msg = 'Running Test Five: zscore p-value is "0.05".'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': "0.05"})
    ts.append(['t_5_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: zscore p-value is string of 0.1
    msg = 'Running Test Five: zscore p-value is "0.1".'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': "0.1"})
    ts.append(['t_5_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: zscore p-value is string of 1.0 (not supported)
    msg = 'Running Test Five: zscore p-value is "1.0" (not supported).'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': "1.0"})
    ts.append(['t_5_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: zscore p-value is float of 0.01
    msg = 'Running Test Five: zscore p-value is 0.01.'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': 0.01})
    ts.append(['t_5_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: zscore p-value is int of 1 (not supported)
    msg = 'Running Test Five: zscore p-value is 1 (not supported).'
    params = inputs.copy()
    params.update({'in_zscore_pvalue': 1})
    ts.append(['t_5_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     
            
    return ts

### Test Five: Run!

In [14]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_five_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Performing test: nc_default_t_5_def.nc.
Performing test: nc_default_t_5_a.nc.
Performing test: nc_default_t_5_b.nc.
Performing test: nc_default_t_5_c.nc.
Performing test: nc_default_t_5_d.nc.
Performing test: nc_default_t_5_e.nc.
Failed to execute. Parameters are not valid.
ERROR 000800: The value is not a member of 0.01 | 0.05 | 0.1.
Failed to execute (GDVSpectra_Likelihood).

Performing test: nc_default_t_5_f.nc.
Performing test: nc_default_t_5_g.nc.
Failed to execute. Parameters are not valid.
ERROR 000800: The value is not a member of 0.01 | 0.05 | 0.1.
Failed to execute (GDVSpectra_Likelihood).

All tests applied to corruptor NetCDF.




### Test Six: IVT Standardisation

In [15]:
def build_test_six_funcs(in_nc, temp_nc):
    """sets up test six functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': None,                  # upper quantile for standardisation
        'in_stand_qlower': None,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default ivt lower and upper
    msg = 'Running Test Six: default IVT lower, upper input (0.05, 0.99).'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.05, 'in_stand_qupper': 0.99})
    ts.append(['t_6_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: ivt lower and upper "", ""
    msg = 'Running Test Six: IVT lower, upper input "", "".'
    params = inputs.copy()
    params.update({'in_stand_qlower': '', 'in_stand_qupper': ''})
    ts.append(['t_6_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: ivt lower and upper 0.0, 0.99
    msg = 'Running Test Six: IVT lower, upper input 0.0, 0.99.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.0, 'in_stand_qupper': 0.99})
    ts.append(['t_6_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: ivt lower and upper 0.05, 0.0
    msg = 'Running Test Six: IVT lower, upper input 0.0, 0.99.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.05, 'in_stand_qupper': 0.0})
    ts.append(['t_6_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: ivt lower and upper 0.25, 0.80
    msg = 'Running Test Six: IVT lower, upper input 0.25, 0.80.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.25, 'in_stand_qupper': 0.80})
    ts.append(['t_6_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: ivt lower and upper 0.4, 0.6
    msg = 'Running Test Six: IVT lower, upper input 0.4, 0.6.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.4, 'in_stand_qupper': 0.6})
    ts.append(['t_6_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    # func: ivt lower and upper 0.5, 0.5
    msg = 'Running Test Six: IVT lower, upper input 0.5, 0.5.'
    params = inputs.copy()
    params.update({'in_stand_qlower': 0.5, 'in_stand_qupper': 0.5})
    ts.append(['t_6_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])     

    # func: ivt lower and upper "0.2", "0.8"
    msg = 'Running Test Six: IVT lower, upper input "0.2", "0.8".'
    params = inputs.copy()
    params.update({'in_stand_qlower': "0.2", 'in_stand_qupper': "0.8"})
    ts.append(['t_6_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    return ts

### Test Six: Run!

In [19]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_six_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Performing test: nc_default_t_6_def.nc.
Performing test: nc_default_t_6_a.nc.
Performing test: nc_default_t_6_b.nc.
Performing test: nc_default_t_6_c.nc.
Standardisation upper quantile must be higher than lower.
Failed to execute (GDVSpectra_Likelihood).

Performing test: nc_default_t_6_d.nc.
Performing test: nc_default_t_6_e.nc.
Performing test: nc_default_t_6_f.nc.
Standardisation upper quantile must be higher than lower.
Failed to execute (GDVSpectra_Likelihood).

Performing test: nc_default_t_6_g.nc.
All tests applied to corruptor NetCDF.




### Test Seven: Pixel Flags

In [23]:
def build_test_seven_funcs(in_nc, temp_nc):
    """sets up test seven functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': '',                     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default pixel flags
    msg = 'Running Test Seven: pixel flags Valid;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid;Snow;Water'})
    ts.append(['t_7_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: pixel flags is ''
    msg = 'Running Test Seven: pixel flags is "".'
    params = inputs.copy()
    params.update({'in_fmask_flags': ''})
    ts.append(['t_7_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water
    msg = 'Running Test Seven: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'NoData;Valid;Cloud;Shadow;Snow;Water'})
    ts.append(['t_7_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: pixel flags is Valid
    msg = 'Running Test Seven: pixel flags is Valid.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid'})
    ts.append(['t_7_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is Cloud;Shadow
    msg = 'Running Test Seven: pixel flags is Cloud;Shadow.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Cloud;Shadow'})
    ts.append(['t_7_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is Water
    msg = 'Running Test Seven: pixel flags is Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Water'})
    ts.append(['t_7_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is Water;Water
    msg = 'Running Test Seven: pixel flags is Water;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Water;Water'})
    ts.append(['t_7_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 

    # func: pixel flags is water
    msg = 'Running Test Seven: pixel flags is water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'water'})
    ts.append(['t_7_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg]) 
    
    return ts

### Test Seven: Run!

In [24]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_seven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=True)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\yandi_2_ls_90_20_raw_odc.nc
No changes, setting up for default dataset.
Performing test: nc_default_t_7_def.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [1, 2, 3, 9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['mavi', 'ndmi'].
Calculating ind

### Test Eight: Max Cloud Cover

In [25]:
def build_test_eight_funcs(in_nc, temp_nc):
    """sets up test eight functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': '',                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default max cloud cover
    msg = 'Running Test Eight: default max cloud (10).'
    params = inputs.copy()
    params.update({'in_max_cloud': 10})
    ts.append(['t_8_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])

    # func: max cloud cover is 0
    msg = 'Running Test Eight: max cloud cover is 0.'
    params = inputs.copy()
    params.update({'in_max_cloud': 0})
    ts.append(['t_8_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: max cloud cover is 50
    msg = 'Running Test Eight: max cloud cover is 50.'
    params = inputs.copy()
    params.update({'in_max_cloud': 50})
    ts.append(['t_8_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: max cloud cover is "10"
    msg = 'Running Test Eight: max cloud cover is "10".'
    params = inputs.copy()
    params.update({'in_max_cloud': "10"})
    ts.append(['t_8_c.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])  
    
    # func: max cloud cover is "10"
    msg = 'Running Test Eight: max cloud cover is "10".'
    params = inputs.copy()
    params.update({'in_max_cloud': "10"})
    ts.append(['t_8_d.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])      

    # func: max cloud cover is 150
    msg = 'Running Test Eight: max cloud cover is 150.'
    params = inputs.copy()
    params.update({'in_max_cloud': 150})
    ts.append(['t_8_e.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])   
    
    # func: max cloud cover is ''
    msg = 'Running Test Eight: max cloud cover is "".'
    params = inputs.copy()
    params.update({'in_max_cloud': ''})
    ts.append(['t_8_f.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])   

    # func: max cloud cover is None
    msg = 'Running Test Eight: max cloud cover is None.'
    params = inputs.copy()
    params.update({'in_max_cloud': None})
    ts.append(['t_8_g.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])  
    
    return ts

### Test Eight: Run!

In [26]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eight_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Performing test: nc_default_t_8_def.nc.
Performing test: nc_default_t_8_a.nc.
Performing test: nc_default_t_8_b.nc.
Performing test: nc_default_t_8_c.nc.
Performing test: nc_default_t_8_d.nc.
Performing test: nc_default_t_8_e.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 0 to 100.
Failed to execute (GDVSpectra_Likelihood).

Performing test: nc_default_t_8_f.nc.
Performing test: nc_default_t_8_g.nc.
All tests applied to corruptor NetCDF.




### Test Nine: Interpolate

In [27]:
def build_test_nine_funcs(in_nc, temp_nc):
    """sets up test nine functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': '',                     # interpolate missing pixels
        'in_add_result_to_map': True,             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default interpolate (True)
    msg = 'Running Test Nine: default interpolate (True).'
    params = inputs.copy()
    params.update({'in_interpolate': True})
    ts.append(['t_9_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: interpolate is False
    msg = 'Running Test Nine: interpolate is False.'
    params = inputs.copy()
    params.update({'in_interpolate': False})
    ts.append(['t_9_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: interpolate is None
    msg = 'Running Test Nine: interpolate is None (default to True).'
    params = inputs.copy()
    params.update({'in_interpolate': None})
    ts.append(['t_9_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])       
            
    return ts

### Test Nine: Run!

In [28]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_nine_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Performing test: nc_default_t_9_def.nc.
Performing test: nc_default_t_9_a.nc.
Performing test: nc_default_t_9_b.nc.
All tests applied to corruptor NetCDF.




### Test Ten: Add To Map

In [29]:
def build_test_ten_funcs(in_nc, temp_nc):
    """sets up test ten functions"""
    
    # set default params for tool
    inputs = {
        'in_nc_file': '',                         # input nc (i.e. temp nc)
        'out_likelihood_nc_file': '',             # output nc (i.e. t1a nc)
        'in_wet_months': '1;2;3',                 # wet months 
        'in_dry_months': '9;10;11',               # dry months 
        'in_veg_idx': 'MAVI',                     # vege index name
        'in_mst_idx': 'NDMI',                     # moisture index name       
        'in_zscore_pvalue': None,                 # zscore pvalue
        'in_stand_qupper': 0.99,                  # upper quantile for standardisation
        'in_stand_qlower': 0.05,                  # lower quantile for standardisation
        'in_fmask_flags': 'Valid;Snow;Water',     # fmask flag values
        'in_max_cloud': 10,                       # max cloud percentage
        'in_interpolate': True,                   # interpolate missing pixels
        'in_add_result_to_map': '',             # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default add result to map (True)
    msg = 'Running Test Ten: default add result to map (True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': True})
    ts.append(['t_10_def.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])
    
    # func: add result to map is False
    msg = 'Running Test Ten: add result to map is False.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': False})
    ts.append(['t_10_a.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])    

    # func: add result to map is None
    msg = 'Running Test Ten: add result to map is None (default to True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': None})
    ts.append(['t_10_b.nc', arcpy.GDVSpectra_Likelihood_toolbox , params, msg])       
            
    return ts

### Test Ten: Run!

In [30]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_ten_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_coors=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Performing test: nc_default_t_10_def.nc.
Performing test: nc_default_t_10_a.nc.
Performing test: nc_default_t_10_b.nc.
All tests applied to corruptor NetCDF.


